In [ ]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np
import matplotlib.pyplot as plt
from dedalus import public as d3
import pathlib
import glob
import h5py
from scipy.fft import fft, fftfreq

import plotting_setup

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# For IVP I, use `sim_name = 'sim28'`
# For IVP II, use `sim_name = 'sim32'`

sim_name = 'sim28'

In [ ]:
IVP_dir = pathlib.Path('..').joinpath('IVP')
params_path = IVP_dir.joinpath('params').joinpath(sim_name+'.json')
file_dir = ((IVP_dir.joinpath('data')).joinpath(sim_name)).joinpath(sim_name+'-last.hdf5')

figure_path = IVP_dir.joinpath("figures")
if not figure_path.exists():
    figure_path.mkdir()

# Load data
IVP_dict = {}
with h5py.File(file_dir, "r") as f:
    for key in list(f.keys()):
        IVP_dict[key] = f[key][()]

with open(params_path) as f: 
    params_IVP = json.load(f)

# IVP_dict
p_IVP = IVP_dict['p']
u_IVP = IVP_dict['u']
v_IVP = IVP_dict['v']
w_IVP = IVP_dict['w']
rho_IVP = IVP_dict['rho']
Bx_IVP = IVP_dict['Bx']
By_IVP = IVP_dict['By']
Bz_IVP = IVP_dict['Bz']
x_IVP = IVP_dict['x']
Z_IVP = IVP_dict['Z']
Fr = params_IVP['Fr']

Nx = params_IVP['Nx']
x = x_IVP

xx_IVP, ZZ_IVP = np.meshgrid(x_IVP,Z_IVP,indexing='ij')

Lx = params_IVP['Lx']
LZ = params_IVP['LZ']

### Check that sim is equilibrated

In [ ]:
time_series_pattern = ((IVP_dir.joinpath('data')).joinpath(sim_name)).joinpath(sim_name+"-timeseries*.hdf5")
time_series_path = glob.glob(str(time_series_pattern))[0]

# Load data
timeseries_dict = {}
with h5py.File(time_series_path, "r") as f:
    for key in list(f.keys()):
        timeseries_dict[key] = f[key][()]

In [ ]:
for scalar in ['E','ME','KE']:
    plt.plot(timeseries_dict['t'],timeseries_dict[scalar],label=scalar)
plt.legend()
plt.xlabel('$t$')
plt.grid(alpha=0.5)
# plt.yscale('log')

In [ ]:
t99 = timeseries_dict['t'].flatten()[timeseries_dict['E'].flatten() >= 0.99*timeseries_dict['E'][-1].flatten()][0]
print((timeseries_dict['t'][-1] - t99)/(2*np.pi))

### Check that sim is resolved

In [ ]:
Z_prof_val = 0.091 + 0.025
v_prof = v_IVP[:,np.argmin(np.abs(Z_IVP-Z_prof_val))]
fig,axs = plt.subplots(1,2,figsize=(10,5))
for ax in axs:
    ax.plot(x_IVP,v_prof.real)
    ax.grid()
    ax.set_xlabel("$x/L$")
    ax.set_ylabel(r"$\Re\{v\}$")
axs[1].set_xlim(0.25-0.05,0.25+0.05)
plt.suptitle(rf"$z/L = {Z_prof_val:.3f}$")

In [ ]:
def spectral_coeff(c,Lx):
    Nx = len(c)
    coords =d3.CartesianCoordinates('x')
    dist = d3.Distributor(coords, dtype=np.complex128)
    xbasis = d3.ComplexFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=3/2)
    cfield = dist.Field(name='c', bases=xbasis)
    cfield['g'] = c
    coeffs = cfield['c']
    coeffs_shifted = np.zeros(len(coeffs),dtype=np.complex128)
    coeffs_shifted[0:Nx//2] = coeffs[Nx//2:]
    coeffs_shifted[Nx//2:] = coeffs[:Nx//2]
    wavenumbers = 2*np.pi/Lx * np.arange(-Nx//2,Nx//2,1)
    return coeffs_shifted,wavenumbers

In [ ]:
v_prof_coeffs,wavenumbers = spectral_coeff(v_prof,Lx)
from scipy.signal import find_peaks

v_prof_coeffs_odd = v_prof_coeffs[Nx//2+1::2]
wavelengths_odd = 2*np.pi/wavenumbers[Nx//2+1::2]
peaks, _ = find_peaks(np.abs(v_prof_coeffs_odd), prominence=1e-4)
plt.loglog(wavelengths_odd,np.abs(v_prof_coeffs_odd))
plt.plot(wavelengths_odd[peaks], np.abs(v_prof_coeffs_odd)[peaks], "x")
print(np.min(wavelengths_odd[peaks]))
plt.xlabel(r"$2\pi/k$")
plt.ylabel("Absolute value of Fourier coefficient")

### Plot

In [ ]:
def sin_cos_decomp(c,Lx,LZ):
    Nx,NZ = c.shape
    coords_real = d3.CartesianCoordinates('x', 'Z')
    dist_real = d3.Distributor(coords_real, dtype=np.float64)
    xbasis_real = d3.RealFourier(coords_real['x'], size=Nx, bounds=(0, Lx), dealias=3/2)
    Zbasis_real = d3.RealFourier(coords_real['Z'], size=NZ, bounds=(0, LZ), dealias=3/2)
    # xgrid_real, Zgrid_real = dist_real.local_grids(xbasis_real, Zbasis_real)

    creal_cos = dist_real.Field(name='creal_cos', bases=(xbasis_real, Zbasis_real))
    creal_sin = dist_real.Field(name='creal_sin', bases=(xbasis_real, Zbasis_real))
    cimag_cos = dist_real.Field(name='cimag_cos', bases=(xbasis_real, Zbasis_real))
    cimag_sin = dist_real.Field(name='cimag_sin', bases=(xbasis_real, Zbasis_real))

    creal_cos['g'] = c.real
    creal_sin['g'] = c.real
    cimag_cos['g'] = c.imag
    cimag_sin['g'] = c.imag

    creal_cos['c'][1::2,:] = 0
    creal_sin['c'][::2,:] = 0
    cimag_cos['c'][1::2,:] = 0
    cimag_sin['c'][::2,:] = 0

    c_cos = creal_cos['g'] + 1j*cimag_cos['g']
    c_sin = creal_sin['g'] + 1j*cimag_sin['g']

    return c_sin, c_cos

In [ ]:
p_sin, p_cos = sin_cos_decomp(p_IVP,Lx,LZ)
u_sin, u_cos = sin_cos_decomp(u_IVP,Lx,LZ)
v_sin, v_cos = sin_cos_decomp(v_IVP,Lx,LZ)
w_sin, w_cos = sin_cos_decomp(w_IVP,Lx,LZ)
Bx_sin, Bx_cos = sin_cos_decomp(Bx_IVP,Lx,LZ)
By_sin, By_cos = sin_cos_decomp(By_IVP,Lx,LZ)

In [ ]:
strd = 1
interpolation = "hanning"
interpolation_stage = "rgba"
dpi = 500
save_fig = True
hires = False

if hires:
    dpi_to_use = 6000
    save_path = figure_path.joinpath(f"{sim_name}_sincos_dpi{dpi_to_use}.jpg")
else:
    dpi_to_use = dpi
    save_path = figure_path.joinpath(f"{sim_name}_sincos_dpi{dpi_to_use}.pdf")

# Matplotlib issue with pre-existing pdf
try:
    pathlib.Path.unlink(save_path)
except OSError:
    pass
import time
time.sleep(5)

gridspec = dict(hspace=0.0, width_ratios=[1, 1, 1, 0.4, 1, 1, 1])
fig, axs = plt.subplots(1,7,figsize = (7.52,7.52/8*5),gridspec_kw = gridspec)
axs[3].set_visible(False)
title_fontsize = 12

if hires:
    interpolation = 'none'

vmin = np.max(np.abs(u_IVP.real))
axs[0].imshow(u_IVP.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
axs[0].set_title(r"$u_{\text{IVP}}$")
axs[2].imshow(u_cos.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
axs[2].set_title(r"$u_{\text{IVP}}$ (cos)")
axs[1].imshow(u_sin.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
axs[1].set_title(r"$u_{\text{IVP}}$ (sin)")

vmin = np.max(np.abs(v_IVP.real))
axs[4].imshow(v_IVP.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
axs[4].set_title(r"$v_{\text{IVP}}$")
axs[5].imshow(v_cos.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
axs[5].set_title(r"$v_{\text{IVP}}$ (cos)")
axs[6].imshow(v_sin.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
axs[6].set_title(r"$v_{\text{IVP}}$ (sin)")

panel_label = ['($a$)','($b$)','($c$)','.','($d$)','($e$)','($f$)']
for k in [0,1,2,4,5,6]:
    ax = axs[k]
    if k in [1,2,5,6]:
        ax.set_yticks([])
    else:
        ax.set_ylabel('$z/L$', labelpad=-5)
    ax.text(0.1, 1.11, panel_label[k], transform=ax.transAxes, fontsize=14, va='top', ha='right')
    
    ax.axhline(params_IVP['Z0'],linestyle='dotted',color='k',lw=1.5)
    xlims = ax.get_xlim()
    ylims = ax.get_ylim()
    ax.fill_between([xlims[0],xlims[1]],[params_IVP['s'],params_IVP['s']], facecolor="none", hatch="//////", edgecolor="k", linewidth=0.5)
    ax.fill_between([xlims[0],xlims[1]],[LZ-params_IVP['s'],LZ-params_IVP['s']],[LZ,LZ], facecolor="none", hatch="//////", edgecolor="k", linewidth=0.5)
    ax.set_xticks([0,0.5,1],['0','0.5','1'])
    ax.set_xlabel("$x/L$")
    ax.set_aspect(4.5 * 1/0.25)

if save_fig:
    plt.savefig(save_path,dpi=dpi_to_use)

plt.show()

In [ ]:
plt.imshow(v_IVP.T.real[::strd,::strd],cmap='RdBu_r',vmin=(-vmin,vmin),extent=[0,Lx,0,LZ],origin="lower",interpolation=interpolation,interpolation_stage=interpolation_stage)
plt.axis('off')
plt.gca().set_aspect(4)
save_path = figure_path.joinpath(f"{sim_name}_v_image.jpg")
plt.tight_layout()
plt.savefig(save_path,dpi=1000,bbox_inches='tight')

In [ ]:
def fieldline(x,contourlevel):
    return np.log(-(np.sin(kb*x)/(contourlevel*kb)))/kb

kb = params_IVP['kb']
Z_shift = 0 
contlevels = [-0.02,-0.06,-0.1,-0.14,0.02,0.06,0.1,0.14]
x_magn = np.linspace(0,Lx,3000)
Z_magn = np.linspace(0,LZ,len(x_magn))
Z_shift = 0
fig, axs = plt.subplots(1,1,figsize=(3.5,1.4))
axs = [axs]

for c in contlevels:
    sc = axs[0].scatter(x_magn,Z_shift+fieldline(x_magn,c),c=np.exp(-kb*fieldline(x_magn,c)),s=2,vmin=(0.2,1),cmap="Purples", rasterized=True)

axs[0].set_xlim(0,Lx)
axs[0].set_ylim(Z_shift+0,Z_shift+LZ)
axs[0].set_aspect(1)
xlims = [0,Lx]
axs[0].set_xlabel('$x/L$')
axs[0].set_ylabel('$z/L$',labelpad=-12)
axs[0].set_yticks([0,LZ],['0',f"{LZ:.2f}"])
axs[0].set_xticks([0,0.25,0.5,0.75,1],['0','0.25','0.5','0.75','1'])

for ax in axs:
    ax.axhline(params_IVP['Z0'],linestyle='dotted',color='k',lw=1)
    ax.fill_between([xlims[0],xlims[1]],[params_IVP['s'],params_IVP['s']], facecolor="none", hatch="////////////", edgecolor="k", linewidth=0.5)
    ax.fill_between([xlims[0],xlims[1]],[LZ-params_IVP['s'],LZ-params_IVP['s']],[LZ,LZ], facecolor="none", hatch="////////////", edgecolor="k", linewidth=0.5) 

save_path = figure_path.joinpath("diagram.pdf")
plt.savefig(save_path,dpi=500,bbox_inches='tight')